In [1]:
import sympy as smp
from sympy.interactive.printing import init_printing
import numpy as np
import string

init_printing(use_unicode=True, wrap_line=False)

In [24]:
L, A, E, EI = smp.symbols("L A E EI", real=True)

In [25]:
no_of_spans = 4
P = [smp.symbols("P%d" % i) for i in range(1, no_of_spans + 1)]
g = [smp.symbols("g%d" % i) for i in range(1, no_of_spans + 1)]
q = [smp.symbols("q%d" % i) for i in range(1, no_of_spans + 1)]
print(P, g, q)
g_loads = np.array([3, 3, 3, 3])
q_loads = np.array([5, 5, 5, 5])
P_loads = g_loads + q_loads
print(P_loads)
span_lengths = smp.Array([5, 5.5, 6.5, 3])
# phi = smp.MutableDenseNDimArray(np.zeros((no_of_spans - 1, no_of_spans - 1)))
phi0 = smp.zeros(no_of_spans, 1)
phi = smp.zeros(no_of_spans - 1, no_of_spans - 1)

# phi[0,1] = P*L**3/24
# print(span_lengths, phi, len(phi), phi0)
phi, phi0

[P1, P2, P3, P4] [g1, g2, g3, g4] [q1, q2, q3, q4]
[8 8 8 8]


⎛           ⎡0⎤⎞
⎜⎡0  0  0⎤  ⎢ ⎥⎟
⎜⎢       ⎥  ⎢0⎥⎟
⎜⎢0  0  0⎥, ⎢ ⎥⎟
⎜⎢       ⎥  ⎢0⎥⎟
⎜⎣0  0  0⎦  ⎢ ⎥⎟
⎝           ⎣0⎦⎠

In [26]:
# getting the flexibility coefficient matrices
k = 0
for i in range(no_of_spans - 1):
    if i == 0 or i == no_of_spans - 2:
        if i == 0:
            print("f")
            phi0[i] = (P[i] * span_lengths[i] ** 3 / 24).n()
            phi[i, 0] = ((span_lengths[i] + span_lengths[i + 1]) / 3).n()
            phi[i, 1] = (span_lengths[i + 1] / 6).n()
        if i == no_of_spans - 2:
            print("l")
            phi0[i] = (P[i] * span_lengths[i] ** 3 / 24).n()
            phi0[i + 1] = (P[i + 1] * span_lengths[i + 1] ** 3 / 24).n()
            phi[i, no_of_spans - 3] = (span_lengths[i] / 6).n()
            phi[i, no_of_spans - 2] = ((span_lengths[i] + span_lengths[i + 1]) / 3).n()
    else:
        print("m")
        phi0[i] = (P[i] * span_lengths[i] ** 3 / 24).n()
        phi[i, k] = (span_lengths[i] / 6).n()
        phi[i, k + 1] = ((span_lengths[i] + span_lengths[i + 1]) / 3).n()
        phi[i, k + 2] = (span_lengths[i + 1] / 6).n()
        k += 1
# print(phi, len(phi), "\n", phi0)
phi, phi0

f
m
l


⎛                                                          ⎡5.20833333333333⋅P₁⎤⎞
⎜⎡       3.5         0.916666666666667         0        ⎤  ⎢                   ⎥⎟
⎜⎢                                                      ⎥  ⎢6.93229166666667⋅P₂⎥⎟
⎜⎢0.916666666666667         4.0         1.08333333333333⎥, ⎢                   ⎥⎟
⎜⎢                                                      ⎥  ⎢11.4427083333333⋅P₃⎥⎟
⎜⎣        0          1.08333333333333   3.16666666666667⎦  ⎢                   ⎥⎟
⎝                                                          ⎣     1.125⋅P₄      ⎦⎠

In [27]:
# getting the moment equations in terms of P
k = 0
d = {}
for i in range(no_of_spans):
    phi_P = smp.zeros(no_of_spans - 1, 1)
    if i == 0 or i == no_of_spans - 1:
        if i == 0:
            phi_P[i] = phi0[i]
            # exec(f"case_{string.ascii_uppercase[i]} = phi.inv() * -phi_P")
            d["case_{0}".format(string.ascii_uppercase[i])] = phi.inv() * -phi_P

        if i == no_of_spans - 1:
            phi_P[i - 1] = phi0[i]
            # exec(f"case_{string.ascii_uppercase[i]} = phi.inv() * -phi_P")
            d["case_{0}".format(string.ascii_uppercase[i])] = phi.inv() * -phi_P

    else:
        phi_P[k], phi_P[k + 1] = phi0[i], phi0[i]
        k += 1
        # exec(f"case_{string.ascii_uppercase[i]} = phi.inv() * -phi_P")
        d["case_{0}".format(string.ascii_uppercase[i])] = phi.inv() * -phi_P

case_list = list(d.keys())
print(case_list, len(case_list))

eqns = smp.zeros(no_of_spans - 1, 1)
for cases in case_list:
    eqns = eqns + d[cases]

# smp.pprint(eqns)
eqns

# d

['case_A', 'case_B', 'case_C', 'case_D'] 4


⎡ -1.59350351244762⋅P₁ - 1.58526832480897⋅P₂ + 0.58172506624353⋅P₃ - 0.029740263741681⋅P₄ ⎤
⎢                                                                                         ⎥
⎢ 0.402467956618191⋅P₁ - 1.50965730527483⋅P₂ - 2.22113207111166⋅P₃ + 0.113553734286419⋅P₄ ⎥
⎢                                                                                         ⎥
⎣-0.137686406211486⋅P₁ + 0.516461709699285⋅P₂ - 2.85362587040917⋅P₃ - 0.394110488045354⋅P₄⎦

In [28]:
# span loading for envelope
loads_Negative_BM = {}
for i in range(1, no_of_spans):
    loads_Negative_BM["support_{0}".format(string.ascii_uppercase[i])] = [i - 1, i]

    for forward in range(i + 2, no_of_spans, 2):
        loads_Negative_BM["support_{0}".format(string.ascii_uppercase[i])].append(
            forward
        )

    for backward in range(i - 3, -1, -2):
        loads_Negative_BM["support_{0}".format(string.ascii_uppercase[i])].append(
            backward
        )
print(loads_Negative_BM)

loads_Positive_BM = {}
for i in range(1, no_of_spans + 1):
    loads_Positive_BM["span_{0}".format(i)] = [i - 1]

    for forward in range(i + 1, no_of_spans, 2):
        # print(forward)
        loads_Positive_BM["span_{0}".format(i)].append(forward)

    for backward in range(i - 3, -1, -2):
        loads_Positive_BM["span_{0}".format(i)].append(backward)
print(loads_Positive_BM)

{'support_B': [0, 1, 3], 'support_C': [1, 2], 'support_D': [2, 3, 0]}
{'span_1': [0, 2], 'span_2': [1, 3], 'span_3': [2, 0], 'span_4': [3, 1]}


In [29]:
# negative Bending moments
moments_N = smp.zeros(no_of_spans - 1, 1)
for i in range(len(eqns)):
    moments_N[i] = eqns[i]

j = 0
for keys in loads_Negative_BM.keys():
    # print(moments_N)
    for i in range(len(P)):
        if i in loads_Negative_BM[keys]:
            moments_N[j] = moments_N[j].subs(P[i], P_loads[i])
        else:
            moments_N[j] = moments_N[j].subs(P[i], g_loads[i])
    j += 1

print(moments_N)
# print(eqns)

Matrix([[-23.9229216092556], [-28.2982499383781], [-25.5339969882302]])


In [30]:
# detailed negative Bending moments
moments_end_N = smp.zeros(no_of_spans - 1, 1)
moments_negative = {}
j = 1
for keys in loads_Negative_BM.keys():
    moments_negative["support_{0}".format(string.ascii_uppercase[j])] = []
    for k in range(len(eqns)):
        moments_end_N[k] = eqns[k]
    for i in range(len(P)):
        for n in range(len(moments_end_N)):
            if i in loads_Negative_BM[keys]:
                moments_end_N[n] = moments_end_N[n].subs(P[i], P_loads[i])
            else:
                moments_end_N[n] = moments_end_N[n].subs(P[i], g_loads[i])
    moments_negative["support_{0}".format(string.ascii_uppercase[j])] = moments_end_N[:]
    j += 1
moments_negative

{'support_B': [-23.9229216092556, -14.6124811282968, -8.68355908768795],
 'support_C': [-12.8980773970914, -28.2982499383781, -20.2927039684496],
 'support_D': [-13.0879546539931, -18.1698549574809, -25.5339969882302]}

In [31]:
# positive Bending moments
moments_end = smp.zeros(no_of_spans - 1, 1)
moments_positive = {}
j = 1
for keys in loads_Positive_BM.keys():
    moments_positive["span_{0}".format(j)] = []
    for k in range(len(eqns)):
        moments_end[k] = eqns[k]
    for i in range(len(P)):
        for n in range(len(moments_end)):
            if i in loads_Positive_BM[keys]:
                moments_end[n] = moments_end[n].subs(P[i], P_loads[i])
            else:
                moments_end[n] = moments_end[n].subs(P[i], g_loads[i])
    moments_positive["span_{0}".format(j)] = moments_end[:]
    j += 1

moments_positive

{'span_1': [-12.9392533352847, -18.7376236289130, -23.5634445480034],
 'span_2': [-15.9554040470175, -16.6248209113877, -7.99512705663051],
 'span_3': [-12.9392533352847, -18.7376236289130, -23.5634445480034],
 'span_4': [-15.9554040470175, -16.6248209113877, -7.99512705663051]}

In [32]:
envelope_loads = loads_Negative_BM | loads_Positive_BM
print(envelope_loads)
envelope_moments = moments_negative | moments_positive
envelope_moments

{'support_B': [0, 1, 3], 'support_C': [1, 2], 'support_D': [2, 3, 0], 'span_1': [0, 2], 'span_2': [1, 3], 'span_3': [2, 0], 'span_4': [3, 1]}


{'support_B': [-23.9229216092556, -14.6124811282968, -8.68355908768795],
 'support_C': [-12.8980773970914, -28.2982499383781, -20.2927039684496],
 'support_D': [-13.0879546539931, -18.1698549574809, -25.5339969882302],
 'span_1': [-12.9392533352847, -18.7376236289130, -23.5634445480034],
 'span_2': [-15.9554040470175, -16.6248209113877, -7.99512705663051],
 'span_3': [-12.9392533352847, -18.7376236289130, -23.5634445480034],
 'span_4': [-15.9554040470175, -16.6248209113877, -7.99512705663051]}

In [33]:
# using envelope as whole, iterating the beam right to left
keys_envelope_loads = list(envelope_loads.keys())
keys_envelope_moments = list(envelope_moments.keys())
# total_shear_points_R2L = smp.zeros(len(keys_envelope_moments), no_of_spans + 1)
shear_points_R2L = {}
for i in range(no_of_spans + 1):
    # moments_positive["span_{0}".format(j)] = moments_end[:]
    shear_points_R2L["SF_{0}".format(string.ascii_uppercase[i])] = []

k = len(keys_envelope_moments) - 1
for i in range(len(keys_envelope_moments) - 1, -1, -1):
    n = no_of_spans - 2  # referring to support moments
    for j in range(no_of_spans, -1, -1):
        if j == no_of_spans:
            # print("got here")
            if (j - 1) in envelope_loads[keys_envelope_loads[k]]:
                Q = P_loads[j - 1]  # P_loads[j - 1]
            else:
                Q = g_loads[j - 1]  # g_loads[j - 1]
        elif j == 0:
            if j in envelope_loads[keys_envelope_loads[k]]:
                Q = P_loads[j]  # P[j]  # P_loads[j]
            else:
                Q = g_loads[j]
        else:
            if (j - 1) not in envelope_loads[keys_envelope_loads[k]]:
                Q = g_loads[j - 1]  # g_loads[j]
            else:
                Q = P_loads[j - 1]

        if j == 0 or j == no_of_spans:
            if j == 0:
                shear_points_R2L["SF_{0}".format(string.ascii_uppercase[j])].append(
                    (Q * span_lengths[j] / 2)
                    + (envelope_moments[keys_envelope_moments[k]][0] / span_lengths[j])
                )

            if j == no_of_spans:
                shear_points_R2L["SF_{0}".format(string.ascii_uppercase[j])].append(
                    (Q * span_lengths[j - 1] / 2)
                    + (
                        envelope_moments[keys_envelope_moments[k]][n]
                        / span_lengths[j - 1]
                    )
                )
        else:
            if j == 1:
                # print(j, n)
                shear_points_R2L["SF_{0}".format(string.ascii_uppercase[j])].append(
                    (Q * span_lengths[0] / 2)
                    - (envelope_moments[keys_envelope_moments[k]][0] / span_lengths[0])
                )
                n -= 1

            else:
                # print(j, n)
                shear_points_R2L["SF_{0}".format(string.ascii_uppercase[j])].append(
                    (
                        (Q * span_lengths[j - 1] / 2)
                        - (
                            envelope_moments[keys_envelope_moments[k]][n]
                            / span_lengths[j - 1]
                        )
                        + (
                            envelope_moments[keys_envelope_moments[k]][n - 1]
                            / span_lengths[j - 1]
                        )
                    )
                )
                n -= 1
    k -= 1
shear_points_R2L

{'SF_A': [4.30891919059650,
  17.4121493329431,
  4.30891919059650,
  17.4121493329431,
  17.3824090692014,
  4.92038452058171,
  15.2154156781489],
 'SF_B': [10.6910808094035,
  22.5878506670569,
  10.6910808094035,
  22.5878506670569,
  22.6175909307986,
  10.0796154794183,
  24.7845843218511],
 'SF_C': [22.1217121571582,
  9.30424914429605,
  22.1217121571582,
  9.30424914429605,
  9.17398187336142,
  24.8000313711430,
  20.3071926398257],
 'SF_D': [8.42235479157581,
  26.7424339875524,
  8.42235479157581,
  26.7424339875524,
  27.1329449278076,
  24.7683775430879,
  8.83785814759864],
 'SF_E': [9.33495764778983,
  -3.35448151600115,
  9.33495764778983,
  -3.35448151600115,
  3.48866767058993,
  -2.26423465614986,
  9.10548030410402]}

In [34]:
# using envelope as whole, iterating the beam left to right
keys_envelope_loads = list(envelope_loads.keys())
keys_envelope_moments = list(envelope_moments.keys())
# total_shear_points_L2R = smp.zeros(len(keys_envelope_moments), no_of_spans + 1)
shear_points_L2R = {}
for i in range(no_of_spans + 1):
    # moments_positive["span_{0}".format(j)] = moments_end[:]
    shear_points_L2R["SF_{0}".format(string.ascii_uppercase[i])] = []

k = 0  # referring to load combinations
l = 0
for i in range(len(keys_envelope_moments)):
    # print(i,k)
    n = 0  # referring to support moments
    # m = 1  # referring to span lengths
    for j in range(no_of_spans + 1):
        if j == no_of_spans:
            # print("got here")
            if (j - 1) in envelope_loads[keys_envelope_loads[k]]:
                Q = P_loads[j - 1]
            else:
                Q = g_loads[j - 1]
        elif j not in envelope_loads[keys_envelope_loads[k]]:
            Q = g_loads[j]
        else:
            Q = P_loads[j]

        if j == 0 or j == no_of_spans:
            if j == 0:
                shear_points_L2R["SF_{0}".format(string.ascii_uppercase[j])].append(
                    (Q * span_lengths[j] / 2)
                    + (envelope_moments[keys_envelope_moments[k]][n] / span_lengths[j])
                )
                x = shear_points_L2R["SF_{0}".format(string.ascii_uppercase[j])][l] / Q
                l += 1
                print(x)
            if j == no_of_spans:
                shear_points_L2R["SF_{0}".format(string.ascii_uppercase[j])].append(
                    (Q * span_lengths[j - 1] / 2)
                    + (
                        envelope_moments[keys_envelope_moments[k]][
                            len(span_lengths) - 2
                        ]
                        / span_lengths[j - 1]
                    )
                )
        else:
            if j == no_of_spans - 1:
                # print(m,j)
                shear_points_L2R["SF_{0}".format(string.ascii_uppercase[j])].append(
                    (Q * span_lengths[j] / 2)
                    - (envelope_moments[keys_envelope_moments[k]][n] / span_lengths[j])
                )
                # m += 1
                n += 1

            else:
                # print(m, j)
                shear_points_L2R["SF_{0}".format(string.ascii_uppercase[j])].append(
                    (Q * span_lengths[j] / 2)
                    - (envelope_moments[keys_envelope_moments[k]][n] / span_lengths[j])
                    + (
                        envelope_moments[keys_envelope_moments[k]][n + 1]
                        / span_lengths[j]
                    )
                )
                # m += 1
                n += 1
    k += 1

shear_points_L2R

1.90192695976861
1.64012817352724
2.17280113365017
2.17651866661788
1.43630639686550
2.17651866661788
1.43630639686550


{'SF_A': [15.2154156781489,
  4.92038452058171,
  17.3824090692014,
  17.4121493329431,
  4.30891919059650,
  17.4121493329431,
  4.30891919059650],
 'SF_B': [23.6928073601743,
  19.1999686288570,
  7.32601812663858,
  7.19575085570395,
  21.8782878428418,
  7.19575085570395,
  21.8782878428418],
 'SF_C': [10.6621418524014,
  27.2316224569121,
  24.8670550721924,
  25.2575660124476,
  11.0776452084242,
  25.2575660124476,
  11.0776452084242],
 'SF_D': [14.8945196958960,
  11.2642346561499,
  20.5113323294101,
  12.3544815160012,
  14.6650423522102,
  12.3544815160012,
  14.6650423522102],
 'SF_E': [9.10548030410402,
  -2.26423465614986,
  3.48866767058993,
  -3.35448151600115,
  9.33495764778983,
  -3.35448151600115,
  9.33495764778983]}

In [35]:
# test_non_repeating_shear_values - success
# using envelope as whole, iterating the beam left to right
keys_envelope_loads = list(envelope_loads.keys())
keys_envelope_moments = list(envelope_moments.keys())
# total_shear_points_L2R = smp.zeros(len(keys_envelope_moments), no_of_spans + 1)
shear_points_L2R = {}
for i in range(no_of_spans + 1):
    # moments_positive["span_{0}".format(j)] = moments_end[:]
    shear_points_L2R["SF_{0}".format(string.ascii_uppercase[i])] = []

k = 0  # referring to load combinations
l = 0
for i in range(len(keys_envelope_moments)):
    for j in range(no_of_spans + 1):
        if j == no_of_spans:
            if (j - 1) in envelope_loads[keys_envelope_loads[k]]:
                Q = P_loads[j - 1]
            else:
                Q = g_loads[j - 1]
        elif j not in envelope_loads[keys_envelope_loads[k]]:
            Q = g_loads[j]
        else:
            Q = P_loads[j]

        if j == 0 or j == no_of_spans:
            if j == 0:
                check = (Q * span_lengths[j] / 2) + (
                    envelope_moments[keys_envelope_moments[k]][j] / span_lengths[j]
                )
                if (
                    check
                    not in shear_points_L2R["SF_{0}".format(string.ascii_uppercase[j])]
                ):
                    shear_points_L2R["SF_{0}".format(string.ascii_uppercase[j])].append(
                        (Q * span_lengths[j] / 2)
                        + (
                            envelope_moments[keys_envelope_moments[k]][j]
                            / span_lengths[j]
                        )
                    )
                    x = (
                        shear_points_L2R["SF_{0}".format(string.ascii_uppercase[j])][l]
                        / Q
                    )
                    l += 1
                    # print(x)
            if j == no_of_spans:
                check = (Q * span_lengths[j - 1] / 2) + (
                    envelope_moments[keys_envelope_moments[k]][len(span_lengths) - 2]
                    / span_lengths[j - 1]
                )
                if (
                    check
                    not in shear_points_L2R["SF_{0}".format(string.ascii_uppercase[j])]
                ):
                    shear_points_L2R["SF_{0}".format(string.ascii_uppercase[j])].append(
                        (Q * span_lengths[j - 1] / 2)
                        + (
                            envelope_moments[keys_envelope_moments[k]][
                                len(span_lengths) - 2
                            ]
                            / span_lengths[j - 1]
                        )
                    )
        else:
            if j == no_of_spans - 1:
                print(j)
                check = (Q * span_lengths[j] / 2) - (
                    envelope_moments[keys_envelope_moments[k]][j - 1] / span_lengths[j]
                )
                if check not in shear_points_L2R["SF_{0}".format(string.ascii_uppercase[j])]:
                    shear_points_L2R["SF_{0}".format(string.ascii_uppercase[j])].append(
                        (Q * span_lengths[j] / 2)
                        - (
                            envelope_moments[keys_envelope_moments[k]][j - 1]
                            / span_lengths[j]
                        )
                    )
                else:
                    continue

            else:
                check = (
                    (Q * span_lengths[j] / 2)
                    - (
                        envelope_moments[keys_envelope_moments[k]][j - 1]
                        / span_lengths[j]
                    )
                    + (envelope_moments[keys_envelope_moments[k]][j] / span_lengths[j])
                )

                if (
                    check
                    not in shear_points_L2R["SF_{0}".format(string.ascii_uppercase[j])]
                ):
                    shear_points_L2R["SF_{0}".format(string.ascii_uppercase[j])].append(
                        (Q * span_lengths[j] / 2)
                        - (
                            envelope_moments[keys_envelope_moments[k]][j - 1]
                            / span_lengths[j]
                        )
                        + (
                            envelope_moments[keys_envelope_moments[k]][j]
                            / span_lengths[j]
                        )
                    )
                else:
                    continue
    k += 1

shear_points_L2R

3
3
3
3
3
3
3


{'SF_A': [15.2154156781489,
  4.92038452058171,
  17.3824090692014,
  17.4121493329431,
  4.30891919059650],
 'SF_B': [23.6928073601743,
  19.1999686288570,
  7.32601812663858,
  7.19575085570395,
  21.8782878428418],
 'SF_C': [10.6621418524014,
  27.2316224569121,
  24.8670550721924,
  25.2575660124476,
  11.0776452084242],
 'SF_D': [14.8945196958960,
  11.2642346561499,
  20.5113323294101,
  12.3544815160012,
  14.6650423522102],
 'SF_E': [9.10548030410402,
  -2.26423465614986,
  3.48866767058993,
  -3.35448151600115,
  9.33495764778983]}

In [36]:
# test_non_repeating_values_R2L
keys_envelope_loads = list(envelope_loads.keys())
keys_envelope_moments = list(envelope_moments.keys())
# total_shear_points_R2L = smp.zeros(len(keys_envelope_moments), no_of_spans + 1)
shear_points_R2L = {}
for i in range(no_of_spans + 1):
    # moments_positive["span_{0}".format(j)] = moments_end[:]
    shear_points_R2L["SF_{0}".format(string.ascii_uppercase[i])] = []
k = len(keys_envelope_moments) - 1
for i in range(len(keys_envelope_moments) - 1, -1, -1):
    n = no_of_spans - 2  # referring to support moments
    for j in range(no_of_spans, -1, -1):
        if j == no_of_spans:
            # print("got here")
            if (j - 1) in envelope_loads[keys_envelope_loads[k]]:
                Q = P_loads[j - 1]  # P_loads[j - 1]
            else:
                Q = g_loads[j - 1]  # g_loads[j - 1]
        elif j == 0:
            if j in envelope_loads[keys_envelope_loads[k]]:
                Q = P_loads[j]  # P[j]  # P_loads[j]
            else:
                Q = g_loads[j]
        else:
            if (j - 1) not in envelope_loads[keys_envelope_loads[k]]:
                Q = g_loads[j - 1]  # g_loads[j]
            else:
                Q = P_loads[j - 1]

        if j == 0 or j == no_of_spans:
            if j == 0:
                check = (Q * span_lengths[j] / 2) + (
                    envelope_moments[keys_envelope_moments[k]][0] / span_lengths[j]
                )
                if (
                    check
                    not in shear_points_R2L["SF_{0}".format(string.ascii_uppercase[j])]
                ):
                    shear_points_R2L["SF_{0}".format(string.ascii_uppercase[j])].append(
                        (Q * span_lengths[j] / 2)
                        + (
                            envelope_moments[keys_envelope_moments[k]][0]
                            / span_lengths[j]
                        )
                    )
                else:
                    continue

            if j == no_of_spans:
                print(j,n)
                check = (Q * span_lengths[j - 1] / 2) + (
                    envelope_moments[keys_envelope_moments[k]][j-2] / span_lengths[j - 1]
                )
                if (
                    check
                    not in shear_points_R2L["SF_{0}".format(string.ascii_uppercase[j])]
                ):
                    shear_points_R2L["SF_{0}".format(string.ascii_uppercase[j])].append(
                        (Q * span_lengths[j - 1] / 2)
                        + (
                            envelope_moments[keys_envelope_moments[k]][j-2]
                            / span_lengths[j - 1]
                        )
                    )
                else:
                    continue
        else:
            if j == 1:
                # print(j, n)
                check = (Q * span_lengths[0] / 2) - (
                    envelope_moments[keys_envelope_moments[k]][0] / span_lengths[0]
                )
                if (
                    check
                    not in shear_points_R2L["SF_{0}".format(string.ascii_uppercase[j])]
                ):
                    shear_points_R2L["SF_{0}".format(string.ascii_uppercase[j])].append(
                        (Q * span_lengths[0] / 2)
                        - (
                            envelope_moments[keys_envelope_moments[k]][0]
                            / span_lengths[0]
                        )
                    )
                else:
                    n -= 1
                    continue

            else:
                # print(j, n)
                check = (
                    (Q * span_lengths[j - 1] / 2)
                    - (
                        envelope_moments[keys_envelope_moments[k]][j-1]
                        / span_lengths[j - 1]
                    )
                    + (
                        envelope_moments[keys_envelope_moments[k]][j-2]
                        / span_lengths[j - 1]
                    )
                )
                if (
                    check
                    not in shear_points_R2L["SF_{0}".format(string.ascii_uppercase[j])]
                ):
                    shear_points_R2L["SF_{0}".format(string.ascii_uppercase[j])].append(
                        (
                            (Q * span_lengths[j - 1] / 2)
                            - (
                                envelope_moments[keys_envelope_moments[k]][j-1]
                                / span_lengths[j - 1]
                            )
                            + (
                                envelope_moments[keys_envelope_moments[k]][j-2]
                                / span_lengths[j - 1]
                            )
                        )
                    )
                else:
                    # n -= 1
                    continue
    k -= 1
shear_points_R2L

4 2
4 2
4 2
4 2
4 2
4 2
4 2


{'SF_A': [4.30891919059650,
  17.4121493329431,
  17.3824090692014,
  4.92038452058171,
  15.2154156781489],
 'SF_B': [10.6910808094035,
  22.5878506670569,
  22.6175909307986,
  10.0796154794183,
  24.7845843218511],
 'SF_C': [22.1217121571582,
  9.30424914429605,
  9.17398187336142,
  24.8000313711430,
  20.3071926398257],
 'SF_D': [8.42235479157581,
  26.7424339875524,
  27.1329449278076,
  24.7683775430879,
  8.83785814759864],
 'SF_E': [9.33495764778983,
  -3.35448151600115,
  3.48866767058993,
  -2.26423465614986,
  9.10548030410402]}

In [37]:
# test merging R2L and L2R - success
keys_envelope_loads = list(envelope_loads.keys())
keys_envelope_moments = list(envelope_moments.keys())
# total_shear_points_L2R = smp.zeros(len(keys_envelope_moments), no_of_spans + 1)
total_shear_points = {}
for i in range(no_of_spans + 1):
    # moments_positive["span_{0}".format(j)] = moments_end[:]
    total_shear_points["SF_{0}".format(string.ascii_uppercase[i])] = []

#iterating the beam from left to right

k = 0  # referring to load combinations
l = 0
for i in range(len(keys_envelope_moments)):
    # print(i)
    for j in range(no_of_spans + 1):
        if j == no_of_spans:
            if (j - 1) in envelope_loads[keys_envelope_loads[k]]:
                Q = P_loads[j - 1]
            else:
                Q = g_loads[j - 1]
        elif j not in envelope_loads[keys_envelope_loads[k]]:
            Q = g_loads[j]
        else:
            Q = P_loads[j]

        if j == 0 or j == no_of_spans:
            if j == 0:
                check = (Q * span_lengths[j] / 2) + (
                    envelope_moments[keys_envelope_moments[k]][j] / span_lengths[j]
                )
                if (
                    check
                    not in total_shear_points["SF_{0}".format(string.ascii_uppercase[j])]
                ):
                    total_shear_points["SF_{0}".format(string.ascii_uppercase[j])].append(
                        (Q * span_lengths[j] / 2)
                        + (
                            envelope_moments[keys_envelope_moments[k]][j]
                            / span_lengths[j]
                        )
                    )
                    x = (
                        total_shear_points["SF_{0}".format(string.ascii_uppercase[j])][l]
                        / Q
                    )
                    l += 1
                    # print(x)
            if j == no_of_spans:
                check = (Q * span_lengths[j - 1] / 2) + (
                    envelope_moments[keys_envelope_moments[k]][len(span_lengths) - 2]
                    / span_lengths[j - 1]
                )
                if (
                    check
                    not in total_shear_points["SF_{0}".format(string.ascii_uppercase[j])]
                ):
                    total_shear_points["SF_{0}".format(string.ascii_uppercase[j])].append(
                        (Q * span_lengths[j - 1] / 2)
                        + (
                            envelope_moments[keys_envelope_moments[k]][
                                len(span_lengths) - 2
                            ]
                            / span_lengths[j - 1]
                        )
                    )
        else:
            if j == no_of_spans - 1:
                # print(j)
                check = (Q * span_lengths[j] / 2) - (
                    envelope_moments[keys_envelope_moments[k]][j - 1] / span_lengths[j]
                )
                if (
                    check
                    not in total_shear_points["SF_{0}".format(string.ascii_uppercase[j])]
                ):
                    total_shear_points["SF_{0}".format(string.ascii_uppercase[j])].append(
                        (Q * span_lengths[j] / 2)
                        - (
                            envelope_moments[keys_envelope_moments[k]][j - 1]
                            / span_lengths[j]
                        )
                    )
                else:
                    continue

            else:
                check = (
                    (Q * span_lengths[j] / 2)
                    - (
                        envelope_moments[keys_envelope_moments[k]][j - 1]
                        / span_lengths[j]
                    )
                    + (envelope_moments[keys_envelope_moments[k]][j] / span_lengths[j])
                )

                if (
                    check
                    not in total_shear_points["SF_{0}".format(string.ascii_uppercase[j])]
                ):
                    total_shear_points["SF_{0}".format(string.ascii_uppercase[j])].append(
                        (Q * span_lengths[j] / 2)
                        - (
                            envelope_moments[keys_envelope_moments[k]][j - 1]
                            / span_lengths[j]
                        )
                        + (
                            envelope_moments[keys_envelope_moments[k]][j]
                            / span_lengths[j]
                        )
                    )
                else:
                    continue
    k += 1

#iterating the beam from right to left
k = len(keys_envelope_moments) - 1
for i in range(len(keys_envelope_moments) - 1, -1, -1):
    n = no_of_spans - 2  # referring to support moments
    for j in range(no_of_spans, -1, -1):
        if j == no_of_spans:
            # print("got here")
            if (j - 1) in envelope_loads[keys_envelope_loads[k]]:
                Q = P_loads[j - 1]  # P_loads[j - 1]
            else:
                Q = g_loads[j - 1]  # g_loads[j - 1]
        elif j == 0:
            if j in envelope_loads[keys_envelope_loads[k]]:
                Q = P_loads[j]  # P[j]  # P_loads[j]
            else:
                Q = g_loads[j]
        else:
            if (j - 1) not in envelope_loads[keys_envelope_loads[k]]:
                Q = g_loads[j - 1]  # g_loads[j]
            else:
                Q = P_loads[j - 1]

        if j == 0 or j == no_of_spans:
            if j == 0:
                check = (Q * span_lengths[j] / 2) + (
                    envelope_moments[keys_envelope_moments[k]][0] / span_lengths[j]
                )
                if (
                    check
                    not in total_shear_points["SF_{0}".format(string.ascii_uppercase[j])]
                ):
                    total_shear_points["SF_{0}".format(string.ascii_uppercase[j])].append(
                        (Q * span_lengths[j] / 2)
                        + (
                            envelope_moments[keys_envelope_moments[k]][0]
                            / span_lengths[j]
                        )
                    )
                else:
                    continue

            if j == no_of_spans:
                # print(j, n)
                check = (Q * span_lengths[j - 1] / 2) + (
                    envelope_moments[keys_envelope_moments[k]][j - 2]
                    / span_lengths[j - 1]
                )
                if (
                    check
                    not in total_shear_points["SF_{0}".format(string.ascii_uppercase[j])]
                ):
                    total_shear_points["SF_{0}".format(string.ascii_uppercase[j])].append(
                        (Q * span_lengths[j - 1] / 2)
                        + (
                            envelope_moments[keys_envelope_moments[k]][j - 2]
                            / span_lengths[j - 1]
                        )
                    )
                else:
                    continue
        else:
            if j == 1:
                # print(j, n)
                check = (Q * span_lengths[0] / 2) - (
                    envelope_moments[keys_envelope_moments[k]][0] / span_lengths[0]
                )
                if (
                    check
                    not in total_shear_points["SF_{0}".format(string.ascii_uppercase[j])]
                ):
                    total_shear_points["SF_{0}".format(string.ascii_uppercase[j])].append(
                        (Q * span_lengths[0] / 2)
                        - (
                            envelope_moments[keys_envelope_moments[k]][0]
                            / span_lengths[0]
                        )
                    )
                else:
                    n -= 1
                    continue

            else:
                # print(j, n)
                check = (
                    (Q * span_lengths[j - 1] / 2)
                    - (
                        envelope_moments[keys_envelope_moments[k]][j - 1]
                        / span_lengths[j - 1]
                    )
                    + (
                        envelope_moments[keys_envelope_moments[k]][j - 2]
                        / span_lengths[j - 1]
                    )
                )
                if (
                    check
                    not in total_shear_points["SF_{0}".format(string.ascii_uppercase[j])]
                ):
                    total_shear_points["SF_{0}".format(string.ascii_uppercase[j])].append(
                        (
                            (Q * span_lengths[j - 1] / 2)
                            - (
                                envelope_moments[keys_envelope_moments[k]][j - 1]
                                / span_lengths[j - 1]
                            )
                            + (
                                envelope_moments[keys_envelope_moments[k]][j - 2]
                                / span_lengths[j - 1]
                            )
                        )
                    )
                else:
                    # n -= 1
                    continue
    k -= 1
print()
for keys in total_shear_points.keys():
    print(keys,':',total_shear_points[keys])
# total_shear_points


SF_A : [15.2154156781489, 4.92038452058171, 17.3824090692014, 17.4121493329431, 4.30891919059650]
SF_B : [23.6928073601743, 19.1999686288570, 7.32601812663858, 7.19575085570395, 21.8782878428418, 10.6910808094035, 22.5878506670569, 22.6175909307986, 10.0796154794183, 24.7845843218511]
SF_C : [10.6621418524014, 27.2316224569121, 24.8670550721924, 25.2575660124476, 11.0776452084242, 22.1217121571582, 9.30424914429605, 9.17398187336142, 24.8000313711430, 20.3071926398257]
SF_D : [14.8945196958960, 11.2642346561499, 20.5113323294101, 12.3544815160012, 14.6650423522102, 8.42235479157581, 26.7424339875524, 27.1329449278076, 24.7683775430879, 8.83785814759864]
SF_E : [9.10548030410402, -2.26423465614986, 3.48866767058993, -3.35448151600115, 9.33495764778983]


In [38]:
# getting shear due to negative bending moments R2L

keys_loads_Negative_BM = list(loads_Negative_BM.keys())

keys_moments_negative = list(moments_negative.keys())

total_shear_points_R2L = smp.zeros(len(keys_moments_negative), no_of_spans + 1)


shear_points_R2L = {}
for i in range(no_of_spans + 1):
    # moments_positive["span_{0}".format(j)] = moments_end[:]
    shear_points_R2L["SF_{0}".format(string.ascii_uppercase[i])] = []



k = len(keys_moments_negative) - 1


for i in range(len(keys_moments_negative) - 1, -1, -1):
    n = no_of_spans - 2  # referring to support moments


    for j in range(no_of_spans, -1, -1):
        if j == no_of_spans:
            # print("got here")


            if (j - 1) in loads_Negative_BM[keys_loads_Negative_BM[k]]:
                Q = P_loads[j - 1]  # P_loads[j - 1]


            else:
                Q = g_loads[j - 1]  # g_loads[j - 1]


        elif j == 0:
            if j in loads_Negative_BM[keys_loads_Negative_BM[k]]:
                Q = P_loads[j]  # P[j]  # P_loads[j]


            else:
                Q = g_loads[j]


        else:
            if (j - 1) not in loads_Negative_BM[keys_loads_Negative_BM[k]]:
                Q = g_loads[j - 1]  # g_loads[j]


            else:
                Q = P_loads[j - 1]


        if j == 0 or j == no_of_spans:
            if j == 0:
                total_shear_points_R2L[i, j] = (Q * span_lengths[j] / 2) + (
                    moments_negative[keys_moments_negative[k]][0] / span_lengths[j]
                )
                shear_points_R2L["SF_{0}".format(string.ascii_uppercase[j])].append(
                    (Q * span_lengths[j] / 2)
                    + (moments_negative[keys_moments_negative[k]][0] / span_lengths[j])
                )

            if j == no_of_spans:
                total_shear_points_R2L[i, j] = (Q * span_lengths[j - 1] / 2) + (
                    moments_negative[keys_moments_negative[k]][n] / span_lengths[j - 1]
                )
                shear_points_R2L["SF_{0}".format(string.ascii_uppercase[j])].append(
                    (Q * span_lengths[j - 1] / 2)
                    + (
                        moments_negative[keys_moments_negative[k]][n]
                        / span_lengths[j - 1]
                    )
                )


        else:
            if j == 1:
                # print(j, n)


                total_shear_points_R2L[i, j] = (Q * span_lengths[0] / 2) - (
                    moments_negative[keys_moments_negative[k]][0] / span_lengths[0]
                )
                shear_points_R2L["SF_{0}".format(string.ascii_uppercase[j])].append(
                    (Q * span_lengths[0] / 2)
                    - (moments_negative[keys_moments_negative[k]][0] / span_lengths[0])
                )


                n -= 1


            else:
                # print(j, n)


                total_shear_points_R2L[i, j] = (
                    (Q * span_lengths[j - 1] / 2)
                    - (
                        moments_negative[keys_moments_negative[k]][n]
                        / span_lengths[j - 1]
                    )

                    + (
                        moments_negative[keys_moments_negative[k]][n - 1]
                        / span_lengths[j - 1]
                    )

                )
                shear_points_R2L["SF_{0}".format(string.ascii_uppercase[j])].append(
                    (
                        (Q * span_lengths[j - 1] / 2)
                        - (
                            moments_negative[keys_moments_negative[k]][n]
                            / span_lengths[j - 1]
                        )
                        + (
                            moments_negative[keys_moments_negative[k]][n - 1]
                            / span_lengths[j - 1]
                        )
                    )
                )


                n -= 1


    k -= 1

for keys in shear_points_R2L.keys():
    print(shear_points_R2L[keys])
total_shear_points_R2L

[17.3824090692014, 4.92038452058171, 15.2154156781489]
[22.6175909307986, 10.0796154794183, 24.7845843218511]
[9.17398187336142, 24.8000313711430, 20.3071926398257]
[27.1329449278076, 24.7683775430879, 8.83785814759864]
[3.48866767058993, -2.26423465614986, 9.10548030410402]


⎡15.2154156781489  24.7845843218511  20.3071926398257  8.83785814759864  9.10548030410402 ⎤
⎢                                                                                         ⎥
⎢4.92038452058171  10.0796154794183  24.800031371143   24.7683775430879  -2.26423465614986⎥
⎢                                                                                         ⎥
⎣17.3824090692014  22.6175909307986  9.17398187336142  27.1329449278076  3.48866767058993 ⎦

In [39]:
# getting shear due to negative bending moments L2R

keys_loads_Negative_BM = list(loads_Negative_BM.keys())
keys_moments_negative = list(moments_negative.keys())
total_shear_points_L2R = smp.zeros(len(keys_moments_negative), no_of_spans + 1)
shear_points_L2R = {}
for i in range(no_of_spans + 1):
    # moments_positive["span_{0}".format(j)] = moments_end[:]
    shear_points_L2R["SF_{0}".format(string.ascii_uppercase[i])] = []

k = 0  # referring to load combinations
l = 0
for i in range(len(keys_moments_negative)):
    # print(i,k)
    n = 0  # referring to support moments
    # m = 1  # referring to span lengths
    for j in range(no_of_spans + 1):
        if j == no_of_spans:
            # print("got here")
            if (j - 1) in loads_Negative_BM[keys_loads_Negative_BM[k]]:
                Q = P_loads[j - 1]
            else:
                Q = g_loads[j - 1]
        elif j not in loads_Negative_BM[keys_loads_Negative_BM[k]]:
            Q = g_loads[j]
        else:
            Q = P_loads[j]

        if j == 0 or j == no_of_spans:
            if j == 0:
                total_shear_points_L2R[i, j] = (Q * span_lengths[j] / 2) + (
                    moments_negative[keys_moments_negative[k]][n] / span_lengths[j]
                )
                shear_points_L2R["SF_{0}".format(string.ascii_uppercase[j])].append(
                    (Q * span_lengths[j] / 2)
                    + (moments_negative[keys_moments_negative[k]][n] / span_lengths[j])
                )
                x = shear_points_L2R["SF_{0}".format(string.ascii_uppercase[j])][l] / Q
                l += 1
                print(x)
            if j == no_of_spans:
                total_shear_points_L2R[i, j] = (Q * span_lengths[j - 1] / 2) + (
                    moments_negative[keys_moments_negative[k]][len(span_lengths) - 2]
                    / span_lengths[j - 1]
                )
                shear_points_L2R["SF_{0}".format(string.ascii_uppercase[j])].append(
                    (Q * span_lengths[j - 1] / 2)
                    + (
                        moments_negative[keys_moments_negative[k]][
                            len(span_lengths) - 2
                        ]
                        / span_lengths[j - 1]
                    )
                )
        else:
            if j == no_of_spans - 1:
                # print(m,j)
                total_shear_points_L2R[i, j] = (Q * span_lengths[j] / 2) - (
                    moments_negative[keys_moments_negative[k]][n] / span_lengths[j]
                )
                shear_points_L2R["SF_{0}".format(string.ascii_uppercase[j])].append(
                    (Q * span_lengths[j] / 2)
                    - (moments_negative[keys_moments_negative[k]][n] / span_lengths[j])
                )
                # m += 1
                n += 1

            else:
                # print(m, j)
                total_shear_points_L2R[i, j] = (
                    (Q * span_lengths[j] / 2)
                    - (moments_negative[keys_moments_negative[k]][n] / span_lengths[j])
                    + (
                        moments_negative[keys_moments_negative[k]][n + 1]
                        / span_lengths[j]
                    )
                )
                shear_points_L2R["SF_{0}".format(string.ascii_uppercase[j])].append(
                    (Q * span_lengths[j] / 2)
                    - (moments_negative[keys_moments_negative[k]][n] / span_lengths[j])
                    + (
                        moments_negative[keys_moments_negative[k]][n + 1]
                        / span_lengths[j]
                    )
                )
                # m += 1
                n += 1
    k += 1

shear_points_L2R

1.90192695976861
1.64012817352724
2.17280113365017


{'SF_A': [15.2154156781489, 4.92038452058171, 17.3824090692014],
 'SF_B': [23.6928073601743, 19.1999686288570, 7.32601812663858],
 'SF_C': [10.6621418524014, 27.2316224569121, 24.8670550721924],
 'SF_D': [14.8945196958960, 11.2642346561499, 20.5113323294101],
 'SF_E': [9.10548030410402, -2.26423465614986, 3.48866767058993]}

In [40]:
keys_loads_Positive_BM = list(loads_Positive_BM.keys())
keys_moments_positive = list(moments_positive.keys())
total_shear_points_positive_L2R = smp.zeros(len(keys_moments_positive), no_of_spans + 1)
shear_points_positive_L2R = {}
for i in range(no_of_spans + 1):
    # moments_positive["span_{0}".format(j)] = moments_end[:]
    shear_points_positive_L2R["SF_{0}".format(string.ascii_uppercase[i])] = []

k = 0  # referring to load combinations
l = 0
for i in range(len(keys_moments_positive)):
    # print(i,k)
    n = 0  # referring to support moments
    # m = 1  # referring to span lengths
    for j in range(no_of_spans + 1):
        if j == no_of_spans:
            # print("got here")
            if (j - 1) in loads_Positive_BM[keys_loads_Positive_BM[k]]:
                Q = P_loads[j - 1]
            else:
                Q = g_loads[j - 1]
        elif j not in loads_Positive_BM[keys_loads_Positive_BM[k]]:
            Q = g_loads[j]
        else:
            Q = P_loads[j]

        if j == 0 or j == no_of_spans:
            if j == 0:
                total_shear_points_positive_L2R[i, j] = (Q * span_lengths[j] / 2) + (
                    moments_positive[keys_moments_positive[k]][n] / span_lengths[j]
                )
                shear_points_positive_L2R[
                    "SF_{0}".format(string.ascii_uppercase[j])
                ].append(
                    (Q * span_lengths[j] / 2)
                    + (moments_positive[keys_moments_positive[k]][n] / span_lengths[j])
                )
                x = (
                    shear_points_positive_L2R[
                        "SF_{0}".format(string.ascii_uppercase[j])
                    ][l]
                    / Q
                )
                l += 1
                print(x)
            if j == no_of_spans:
                total_shear_points_positive_L2R[i, j] = (
                    Q * span_lengths[j - 1] / 2
                ) + (
                    moments_positive[keys_moments_positive[k]][len(span_lengths) - 2]
                    / span_lengths[j - 1]
                )
                shear_points_positive_L2R[
                    "SF_{0}".format(string.ascii_uppercase[j])
                ].append(
                    (Q * span_lengths[j - 1] / 2)
                    + (
                        moments_positive[keys_moments_positive[k]][
                            len(span_lengths) - 2
                        ]
                        / span_lengths[j - 1]
                    )
                )
        else:
            if j == no_of_spans - 1:
                # print(m,j)
                total_shear_points_positive_L2R[i, j] = (Q * span_lengths[j] / 2) - (
                    moments_positive[keys_moments_positive[k]][n] / span_lengths[j]
                )
                shear_points_positive_L2R[
                    "SF_{0}".format(string.ascii_uppercase[j])
                ].append(
                    (Q * span_lengths[j] / 2)
                    - (moments_positive[keys_moments_positive[k]][n] / span_lengths[j])
                )
                # m += 1
                n += 1

            else:
                # print(m, j)
                total_shear_points_positive_L2R[i, j] = (
                    (Q * span_lengths[j] / 2)
                    - (moments_positive[keys_moments_positive[k]][n] / span_lengths[j])
                    + (
                        moments_positive[keys_moments_positive[k]][n + 1]
                        / span_lengths[j]
                    )
                )
                shear_points_positive_L2R[
                    "SF_{0}".format(string.ascii_uppercase[j])
                ].append(
                    (Q * span_lengths[j] / 2)
                    - (moments_positive[keys_moments_positive[k]][n] / span_lengths[j])
                    + (
                        moments_positive[keys_moments_positive[k]][n + 1]
                        / span_lengths[j]
                    )
                )
                # m += 1
                n += 1
    k += 1
shear_points_positive_L2R

2.17651866661788
1.43630639686550
2.17651866661788
1.43630639686550


{'SF_A': [17.4121493329431,
  4.30891919059650,
  17.4121493329431,
  4.30891919059650],
 'SF_B': [7.19575085570395,
  21.8782878428418,
  7.19575085570395,
  21.8782878428418],
 'SF_C': [25.2575660124476,
  11.0776452084242,
  25.2575660124476,
  11.0776452084242],
 'SF_D': [12.3544815160012,
  14.6650423522102,
  12.3544815160012,
  14.6650423522102],
 'SF_E': [-3.35448151600115,
  9.33495764778983,
  -3.35448151600115,
  9.33495764778983]}

In [41]:
keys_loads_Positive_BM = list(loads_Positive_BM.keys())
keys_moments_positive = list(moments_positive.keys())
total_shear_points_positive_R2L = smp.zeros(len(keys_moments_positive), no_of_spans + 1)
shear_points_positive_R2L = {}
for i in range(no_of_spans + 1):
    # moments_positive["span_{0}".format(j)] = moments_end[:]
    shear_points_positive_R2L["SF_{0}".format(string.ascii_uppercase[i])] = []

k = len(keys_moments_positive) - 1
for i in range(len(keys_moments_positive) - 1, -1, -1):
    n = no_of_spans - 2  # referring to support moments
    for j in range(no_of_spans, -1, -1):
        if j == no_of_spans:
            # print("got here")
            if (j - 1) in loads_Positive_BM[keys_loads_Positive_BM[k]]:
                Q = P_loads[j - 1]  # P_loads[j - 1]
            else:
                Q = g_loads[j - 1]  # g_loads[j - 1]
        elif j == 0:
            if j in loads_Positive_BM[keys_loads_Positive_BM[k]]:
                Q = P_loads[j]  # P[j]  # P_loads[j]
            else:
                Q = g_loads[j]
        else:
            if (j - 1) not in loads_Positive_BM[keys_loads_Positive_BM[k]]:
                Q = g_loads[j - 1]  # g_loads[j]
            else:
                Q = P_loads[j - 1]

        if j == 0 or j == no_of_spans:
            if j == 0:
                total_shear_points_positive_R2L[i, j] = (Q * span_lengths[j] / 2) + (
                    moments_positive[keys_moments_positive[k]][0] / span_lengths[j]
                )
                shear_points_positive_R2L[
                    "SF_{0}".format(string.ascii_uppercase[j])
                ].append(
                    (Q * span_lengths[j] / 2)
                    + (moments_positive[keys_moments_positive[k]][0] / span_lengths[j])
                )

            if j == no_of_spans:
                total_shear_points_positive_R2L[i, j] = (
                    Q * span_lengths[j - 1] / 2
                ) + (
                    moments_positive[keys_moments_positive[k]][n] / span_lengths[j - 1]
                )
                shear_points_positive_R2L[
                    "SF_{0}".format(string.ascii_uppercase[j])
                ].append(
                    (Q * span_lengths[j - 1] / 2)
                    + (
                        moments_positive[keys_moments_positive[k]][n]
                        / span_lengths[j - 1]
                    )
                )

        else:
            if j == 1:
                # print(j, n)
                total_shear_points_positive_R2L[i, j] = (Q * span_lengths[0] / 2) - (
                    moments_positive[keys_moments_positive[k]][0] / span_lengths[0]
                )
                shear_points_positive_R2L[
                    "SF_{0}".format(string.ascii_uppercase[j])
                ].append(
                    (Q * span_lengths[0] / 2)
                    - (moments_positive[keys_moments_positive[k]][0] / span_lengths[0])
                )
                n -= 1

            else:
                # print(j, n)
                total_shear_points_positive_R2L[i, j] = (
                    (Q * span_lengths[j - 1] / 2)
                    - (
                        moments_positive[keys_moments_positive[k]][n]
                        / span_lengths[j - 1]
                    )
                    + (
                        moments_positive[keys_moments_positive[k]][n - 1]
                        / span_lengths[j - 1]
                    )
                )
                shear_points_positive_R2L[
                    "SF_{0}".format(string.ascii_uppercase[j])
                ].append(
                    (
                        (Q * span_lengths[j - 1] / 2)
                        - (
                            moments_positive[keys_moments_positive[k]][n]
                            / span_lengths[j - 1]
                        )
                        + (
                            moments_positive[keys_moments_positive[k]][n - 1]
                            / span_lengths[j - 1]
                        )
                    )
                )
                n -= 1
    k -= 1
shear_points_positive_R2L

{'SF_A': [4.30891919059650,
  17.4121493329431,
  4.30891919059650,
  17.4121493329431],
 'SF_B': [10.6910808094035,
  22.5878506670569,
  10.6910808094035,
  22.5878506670569],
 'SF_C': [22.1217121571582,
  9.30424914429605,
  22.1217121571582,
  9.30424914429605],
 'SF_D': [8.42235479157581,
  26.7424339875524,
  8.42235479157581,
  26.7424339875524],
 'SF_E': [9.33495764778983,
  -3.35448151600115,
  9.33495764778983,
  -3.35448151600115]}

In [42]:
# positive bending moments
keys_loads_Positive_BM = list(loads_Positive_BM.keys())
keys_moments_positive = list(moments_positive.keys())

shear_points = {}
x_values = {}
for i in range(no_of_spans + 1):
    # moments_positive["span_{0}".format(j)] = moments_end[:]
    x_values["SF0_{0}".format(string.ascii_uppercase[i])] = []
    shear_points["SF_{0}".format(string.ascii_uppercase[i])] = []

bending_moment = {}
for i in range(no_of_spans):
    bending_moment[
        "BM_{0}{1}".format(string.ascii_uppercase[i], string.ascii_uppercase[i + 1])
    ] = []


# print(bending_moment, x_values)
k = 0
l = smp.zeros(1, len(envelope_moments))  # referring to change the counter for x;

n = 0
for i in range(len(keys_moments_positive)):
    # l = 0  # referring to change the counter for x;
    # n = 0
    for j in range(no_of_spans + 1):
        # print(m1, m2, m3)
        # print(j, n)
        if j == no_of_spans:
            # print("got here")
            if (j - 1) in loads_Positive_BM[keys_loads_Positive_BM[k]]:
                Q = P_loads[j - 1]
            else:
                Q = g_loads[j - 1]
        elif j not in loads_Positive_BM[keys_loads_Positive_BM[k]]:
            Q = g_loads[j]
        else:
            Q = P_loads[j]

        if j == 0 or j == no_of_spans:
            # print(j)
            if j == 0:
                # print(j, Q)
                # For A
                check_SF = (Q * span_lengths[j] / 2) + (
                    moments_positive[keys_moments_positive[k]][j] / span_lengths[j]
                )
                if (
                    check_SF
                    not in shear_points["SF_{0}".format(string.ascii_uppercase[j])]
                ):
                    shear_points["SF_{0}".format(string.ascii_uppercase[j])].append(
                        (Q * span_lengths[j] / 2)
                        + (
                            moments_positive[keys_moments_positive[k]][j]
                            / span_lengths[j]
                        )
                    )
                    x_values["SF0_{0}".format(string.ascii_uppercase[j])].append(
                        shear_points["SF_{0}".format(string.ascii_uppercase[j])][l[j]]
                        / Q
                    )
                    x = (
                        shear_points["SF_{0}".format(string.ascii_uppercase[j])][l[j]]
                        / Q
                    )
                    check_BM = (
                        shear_points["SF_{0}".format(string.ascii_uppercase[j])][l[j]]
                        * x
                        - Q * x**2 / 2
                    )
                    if (
                        check_BM
                        not in bending_moment[
                            "BM_{0}{1}".format(
                                string.ascii_uppercase[j], string.ascii_uppercase[j + 1]
                            )
                        ]
                    ):
                        bending_moment[
                            "BM_{0}{1}".format(
                                string.ascii_uppercase[j], string.ascii_uppercase[j + 1]
                            )
                        ].append(
                            shear_points["SF_{0}".format(string.ascii_uppercase[j])][
                                l[j]
                            ]
                            * x
                            - Q * x**2 / 2
                        )

                # For B
                check_SF = (Q * span_lengths[j] / 2) - (
                    moments_positive[keys_moments_positive[k]][j] / span_lengths[j]
                )
                if (
                    check_SF
                    not in shear_points["SF_{0}".format(string.ascii_uppercase[j + 1])]
                ):
                    shear_points["SF_{0}".format(string.ascii_uppercase[j + 1])].append(
                        (Q * span_lengths[j] / 2)
                        - (
                            moments_positive[keys_moments_positive[k]][j]
                            / span_lengths[j]
                        )
                    )
                    x_values["SF0_{0}".format(string.ascii_uppercase[j + 1])].append(
                        shear_points["SF_{0}".format(string.ascii_uppercase[j + 1])][
                            l[j + 1]
                        ]
                        / Q
                    )
                    # print(x_values)
                    x = (
                        shear_points["SF_{0}".format(string.ascii_uppercase[j + 1])][
                            l[j + 1]
                        ]
                        / Q
                    )

                    check_BM = (
                        shear_points["SF_{0}".format(string.ascii_uppercase[j + 1])][
                            l[j + 1]
                        ]
                        * x
                        - Q * x**2 / 2
                        + moments_positive[keys_moments_positive[k]][j]
                    )

                    if (
                        check_BM
                        not in bending_moment[
                            "BM_{0}{1}".format(
                                string.ascii_uppercase[j], string.ascii_uppercase[j + 1]
                            )
                        ]
                    ):
                        # print(
                        #     "Im here",
                        #     Q,
                        #     shear_points[
                        #         "SF_{0}".format(string.ascii_uppercase[j + 1])
                        #     ][l[j]],
                        #     span_lengths[j],
                        #     check_BM,
                        # )
                        bending_moment[
                            "BM_{0}{1}".format(
                                string.ascii_uppercase[j], string.ascii_uppercase[j + 1]
                            )
                        ].append(
                            shear_points[
                                "SF_{0}".format(string.ascii_uppercase[j + 1])
                            ][l[j]]
                            * x
                            - Q * x**2 / 2
                            + moments_positive[keys_moments_positive[k]][0]
                        )
                    l[j] += 1
                    # l[j + 1] += 1
                    # print(x)
        elif j < 3:
            print(j)
            check_SF = (
                (Q * span_lengths[j] / 2)
                - (moments_positive[keys_moments_positive[k]][j - 1] / span_lengths[j])
                + (moments_positive[keys_moments_positive[k]][j] / span_lengths[j])
            )
            print(check_SF)
            if check_SF not in shear_points["SF_{0}".format(string.ascii_uppercase[j])]:
                shear_points["SF_{0}".format(string.ascii_uppercase[j])].append(
                    (Q * span_lengths[j] / 2)
                    - (
                        moments_positive[keys_moments_positive[k]][j - 1]
                        / span_lengths[j]
                    )
                    + (moments_positive[keys_moments_positive[k]][j] / span_lengths[j])
                )
                x_values["SF0_{0}".format(string.ascii_uppercase[j])].append(
                    shear_points["SF_{0}".format(string.ascii_uppercase[j])][l[j]] / Q
                )
                x = shear_points["SF_{0}".format(string.ascii_uppercase[j])][l[j]] / Q
                check_BM = (
                    shear_points["SF_{0}".format(string.ascii_uppercase[j])][l[j]] * x
                    - Q * x**2 / 2
                    + moments_positive[keys_moments_positive[k]][j - 1]
                )
                if (
                    check_BM
                    not in bending_moment[
                        "BM_{0}{1}".format(
                            string.ascii_uppercase[j], string.ascii_uppercase[j + 1]
                        )
                    ]
                ):
                    bending_moment[
                        "BM_{0}{1}".format(
                            string.ascii_uppercase[j], string.ascii_uppercase[j + 1]
                        )
                    ].append(check_BM)
                    l[j] += 1

            check_SF = (
                (Q * span_lengths[j] / 2)
                - (moments_positive[keys_moments_positive[k]][j-1] / span_lengths[j])
                + (moments_positive[keys_moments_positive[k]][j] / span_lengths[j])
            )
            print(check_SF)
            if check_SF not in shear_points["SF_{0}".format(string.ascii_uppercase[j+1])]:
                shear_points["SF_{0}".format(string.ascii_uppercase[j+1])].append(check_SF)
                x_values["SF0_{0}".format(string.ascii_uppercase[j+1])].append(
                    shear_points["SF_{0}".format(string.ascii_uppercase[j+1])][l[j+1]] / Q
                )
                x = shear_points["SF_{0}".format(string.ascii_uppercase[j+1])][l[j+1]] / Q
                check_BM = (
                    shear_points["SF_{0}".format(string.ascii_uppercase[j+1])][l[j+1]] * x
                    - Q * x**2 / 2
                    + moments_positive[keys_moments_positive[k]][j]
                )
                if (
                    check_BM
                    not in bending_moment[
                        "BM_{0}{1}".format(
                            string.ascii_uppercase[j], string.ascii_uppercase[j + 1]
                        )
                    ]
                ):
                    bending_moment[
                        "BM_{0}{1}".format(
                            string.ascii_uppercase[j], string.ascii_uppercase[j + 1]
                        )
                    ].append(check_BM)
                    l[j] += 1
                    # l[j+1] += 1
    k += 1
print(shear_points)
print(x_values, "\n", bending_moment)
l

1
7.19575085570395
7.19575085570395
2
25.2575660124476
25.2575660124476
1
21.8782878428418
21.8782878428418
2
11.0776452084242
11.0776452084242
1
7.19575085570395
7.19575085570395
2
25.2575660124476
25.2575660124476
1
21.8782878428418
21.8782878428418
2
11.0776452084242
11.0776452084242
{'SF_A': [17.4121493329431, 4.30891919059650], 'SF_B': [22.5878506670569, 7.19575085570395, 10.6910808094035, 21.8782878428418], 'SF_C': [7.19575085570395, 25.2575660124476, 21.8782878428418, 11.0776452084242], 'SF_D': [25.2575660124476, 11.0776452084242], 'SF_E': []}
{'SF0_A': [2.17651866661788, 1.43630639686550], 'SF0_B': [2.82348133338212, 7.52928355568565, 3.56369360313450, 1.33638510117544], 'SF0_C': [2.39858361856798, 0.899468856962993, 2.73478598035522, 7.29276261428059], 'SF0_D': [3.15719575155595, 8.41918867081587], 'SF0_E': []} 
 {'BM_AB': [18.9489340245443, 18.9489340245443, 3.09446409851513, -9.36182089832838], 'BM_BC': [72.0959129575927, -10.1078185660190, -8.81170349244276, 13.291396522002

[2  4  4  0  0  0  0]

In [43]:


    # k += 1
# l
# print(shear_points)
# print(x_values, "\n", bending_moment)

In [44]:
# tmps = "g" + "%d" % 3 + "%d" % 2
tmps = "span_{0}{1}{2}".format(4, 89, "g")
print(tmps)
a = 3
if a != 4:
    print("true")

span_489g
true
